# Gen AI - RAG Notebook

## 🎯 Objectives

The objective of this notebook is to do Retrieval-Augmented Generation thanks to langchain framework

You will learn how to use rag using langgraph

## ⚙️ Setup

- Go to your terminal 
- Run the script `00-init.sh`
- Inside this notebook select the good kernel
- Be sure to have your Cohere API Key as environment variable

In [1]:
!pip install langchain
!pip install -qU langchain-core
!pip install -U langchain-cohere
!pip install -qU langgraph

  Obtaining dependency information for langchain from https://files.pythonhosted.org/packages/36/0e/032de736a8f9b5b5fcfec77bd92831f9f2c8a8b5072289dd1e5cc95e6edc/langchain-0.3.22-py3-none-any.whl.metadata
  Obtaining dependency information for langchain-core<1.0.0,>=0.3.49 from https://files.pythonhosted.org/packages/dd/35/27164f5f23517be8639b518130e6235293dae52c41988790e0b50dd7ba11/langchain_core-0.3.49-py3-none-any.whl.metadata
  Obtaining dependency information for langchain-text-splitters<1.0.0,>=0.3.7 from https://files.pythonhosted.org/packages/d3/85/b7a34b6d34bcc89a2252f5ffea30b94077ba3d7adf72e31b9e04e68c901a/langchain_text_splitters-0.3.7-py3-none-any.whl.metadata
  Obtaining dependency information for langsmith<0.4,>=0.1.17 from https://files.pythonhosted.org/packages/94/44/d062769ce2455a04a1a9f4a75cad3ac120449e849a919142d2c4310606a9/langsmith-0.3.21-py3-none-any.whl.metadata
  Obtaining dependency information for SQLAlchemy<3,>=1.4 from https://files.pythonhosted.org/packages/

In [18]:
import getpass
import os

if not os.environ.get("COHERE_API_KEY"):
  os.environ["COHERE_API_KEY"] = getpass.getpass("Enter API key for Cohere: ")

COHERE_API_KEY = os.environ.get('COHERE_API_KEY')

## RAG

Your goal is to do RAG on the content of this blog: https://lilianweng.github.io/posts/2023-06-23-agent/

To do that, you will need to:

- Use WebBaseLoader of langchain to load the content
- Split the document to have 1000 characters (use RecursiveCharacterTextSplitter)
- Embed the contents thanks to a Cohere embedding model
- Store the vectors in a in-memory vectorstore
- Init a langraph instance
- Ask your question
- Print the sources

#### Load the content

In [8]:
from langchain_community.document_loaders import WebBaseLoader

# Remplace l'URL par celle du blog que tu veux charger
url = "https://lilianweng.github.io/posts/2023-06-23-agent/"

# Création du loader
loader = WebBaseLoader(url)

# Chargement des documents
docs = loader.load()

# Affichage des résultats
print("Number of documents:", len(docs))  # Devrait être 1
display(docs)  # Devrait être une liste d'objets Document

Number of documents: 1


[Document(metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/', 'title': "LLM Powered Autonomous Agents | Lil'Log", 'description': 'Building agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general problem solver.\nAgent System Overview\nIn a LLM-powered autonomous agent system, LLM functions as the agent’s brain, complemented by several key components:\n\nPlanning\n\nSubgoal and decomposition: The agent breaks down large tasks into smaller, manageable subgoals, enabling efficient handling of complex tasks.\nReflection and refinement: The agent can do self-criticism and self-reflection over past actions, learn from mistakes and refine them for future steps, thereby improving the quality of final resu

#### Split the document

In [10]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

"""
    Split the document into smaller chunks.
    Use RecursiveCharacterTextSplitter.
    The output should be a list of Document objects.
"""

# Fractionnement du document
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,  # Taille d'un chunk
    chunk_overlap=100  # Chevauchement entre les chunks
)
all_splits = text_splitter.split_documents(docs)

# Affichage des résultats
print("Number of documents:", len(all_splits))  # Devrait être plus de 30
display(all_splits)  # Devrait être une liste d'objets Document


Number of documents: 67


[Document(metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/', 'title': "LLM Powered Autonomous Agents | Lil'Log", 'description': 'Building agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general problem solver.\nAgent System Overview\nIn a LLM-powered autonomous agent system, LLM functions as the agent’s brain, complemented by several key components:\n\nPlanning\n\nSubgoal and decomposition: The agent breaks down large tasks into smaller, manageable subgoals, enabling efficient handling of complex tasks.\nReflection and refinement: The agent can do self-criticism and self-reflection over past actions, learn from mistakes and refine them for future steps, thereby improving the quality of final resu

#### Embed and store the contents

Create the embeddings

In [11]:
from langchain_cohere import CohereEmbeddings

embeddings = CohereEmbeddings(model="embed-english-v3.0")

Create the in memory vectorstore

In [12]:
from langchain_core.vectorstores import InMemoryVectorStore

vector_store = InMemoryVectorStore(embeddings)

Add chunks in the vectorstore

In [16]:
chunk_vector = vector_store.add_documents(documents=all_splits) # Add the documents to the vector store

#### Get info from the docs 

Create a graph

In [19]:
from langchain import hub
from langchain_core.documents import Document
from langgraph.graph import START, StateGraph
from typing_extensions import List, TypedDict

# Define prompt for question-answering
prompt = hub.pull("rlm/rag-prompt")
from langchain.chat_models import init_chat_model

llm = init_chat_model("command-r-plus", model_provider="cohere")


# Define state for application
class State(TypedDict):
    question: str
    context: List[Document]
    answer: str


# Define application steps
def retrieve(state: State):
    """
        Retrieve relevant documents from the vector store.
        Do a similarity search on the question in the state and retrieve the documents.
    """
    retrieved_docs = vector_store.similarity_search(state["question"])
    return {"context": retrieved_docs}


def generate(state: State):
    docs_content = "\n\n".join(doc.page_content for doc in state["context"])
    messages = prompt.invoke({"question": state["question"], "context": docs_content})
    response = llm.invoke(messages)
    return {"answer": response.content}


# Compile application and test
graph_builder = StateGraph(State).add_sequence([retrieve, generate])
graph_builder.add_edge(START, "retrieve")
graph = graph_builder.compile()

/Users/fabienribeiro/PycharmProjects/pythonProject/.venv/lib/python3.12/site-packages/langsmith/client.py:278: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


#### Ask a question

In [20]:
# Poser une question et obtenir une réponse
question_input = "Quel est le sujet principal de l'article ?"
your_response = graph.invoke({"question": question_input})["answer"]

print(your_response)  # Devrait être une chaîne de caractères

L'article fournit des instructions pour la création d'un jeu Super Mario en Python, en utilisant une architecture MVC et des contrôles clavier.


Display sources

In [21]:
# Poser une question et obtenir une réponse
question_input = "Quel est le sujet principal de l'article ?"
result = graph.invoke({"question": question_input})

your_response = result["answer"]
your_sources = result["context"]  # Récupération des documents sources

print(your_response)  # Devrait être une chaîne de caractères

"""Display the sources used to get the answer"""
display(your_sources)

L'article fournit des instructions pour coder un jeu Super Mario en Python, en utilisant une architecture MVC et des contrôles clavier.


[Document(id='4ed79756-04e5-4b3d-98ce-b7e7b50ffbc7', metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/', 'title': "LLM Powered Autonomous Agents | Lil'Log", 'description': 'Building agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general problem solver.\nAgent System Overview\nIn a LLM-powered autonomous agent system, LLM functions as the agent’s brain, complemented by several key components:\n\nPlanning\n\nSubgoal and decomposition: The agent breaks down large tasks into smaller, manageable subgoals, enabling efficient handling of complex tasks.\nReflection and refinement: The agent can do self-criticism and self-reflection over past actions, learn from mistakes and refine them for future steps, 

## 🧠 Go beyond

### 1. The system prompt

#### 1.1 When is it used ?

Your **system prompt** will be provided to the LLM along with the selected relevant documents and the conversation history **each time** the user will ask a question. 

It should include context about your documents, and guidelines about how to answer the user. Here is an example : 

« You are a Cybersecurity Information Assistant, you are able to answer questions related to .. ​
A L'Oréal employee has a question for you. To formulate your answer, follow these rules:  ​
1) Answer him in the language he speaks to you.    ​
2) Use only the context below to answer his question. ​
3) If the context I've given you doesn't allow you to answer, answer that you don't know.   »     

#### 1.2 Edit System Prompt

**It's your time to play**

In [32]:
from langchain.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_cohere import CohereEmbeddings
from langchain_core.vectorstores import InMemoryVectorStore
from langchain import hub
from langchain_core.documents import Document
from langgraph.graph import START, StateGraph
from langchain.chat_models import init_chat_model
from typing_extensions import List, TypedDict

# Remplace l'URL par celle du blog que tu veux charger
url = "https://lilianweng.github.io/posts/2023-06-23-agent/"

# Création du loader
loader = WebBaseLoader(url)

# Chargement des documents
docs = loader.load()

# Fractionnement du document
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,  # Taille d'un chunk
    chunk_overlap=100  # Chevauchement entre les chunks
)
all_splits = text_splitter.split_documents(docs)

# Création et stockage des embeddings
embeddings = CohereEmbeddings(model="embed-english-v3.0")
vector_store = InMemoryVectorStore(embeddings)
chunk_vector = vector_store.add_documents(documents=all_splits)  # Ajout des documents à la base vectorielle

# Définition du System Prompt
system_prompt = (
    "Your name is PromptGPT."
    "To formulate your answer, follow these rules:"
    "Follow these rules: "
    "1) Answer in the same language as the user. "
    "2) Use only the given context to answer. "
    "3) If the context does not allow answering, say you don't know."
)

# Définition du prompt pour le question-answering
prompt = hub.pull("rlm/rag-prompt")
llm = init_chat_model("command-r-plus", model_provider="cohere")


# Définition de l'état de l'application
class State(TypedDict):
    question: str
    context: List[Document]
    answer: str


# Définition des étapes de l'application
def retrieve(state: State):
    """
        Retrieve relevant documents from the vector store.
        Do a similarity search on the question in the state and retrieve the documents.
    """
    retrieved_docs = vector_store.similarity_search(state["question"])
    return {"context": retrieved_docs}


def generate(state: State):
    """
        Generate an answer using the retrieved context and the LLM.
        Format the retrieved documents into a single string to provide context.
    """
    docs_content = "\n\n".join(doc.page_content for doc in state["context"])
    messages = prompt.invoke({
        "system_prompt": system_prompt,
        "question": state["question"],
        "context": docs_content
    })
    response = llm.invoke(messages)
    return {"answer": response.content}


# Compilation et test de l'application
graph_builder = StateGraph(State).add_sequence([retrieve, generate])
graph_builder.add_edge(START, "retrieve")
graph = graph_builder.compile()

# Poser une question et obtenir une réponse
question_input = "What is your name ?"
result = graph.invoke({"question": question_input})

your_response = result["answer"]
"""your_sources = result["context"]  # Récupération des documents sources"""

print(your_response)  # Devrait être une chaîne de caractères

"""Display the sources used to get the answer
display(your_sources)"""


/Users/fabienribeiro/PycharmProjects/pythonProject/.venv/lib/python3.12/site-packages/langsmith/client.py:278: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


{
    "thoughts": {
        "text": "I don't have a name, but I am an AI assistant designed to help with question-answering tasks.",
        "reasoning": null,
        "plan": null,
        "criticism": null,
        "speak": "I don't have a name."
    },
    "command": null
}


'Display the sources used to get the answer\ndisplay(your_sources)'

### 2. The filtering prompt

An optional filtering step is available by adding a *filtering_prompt*. 
The filtering prompt decides to keep a document as relevant or not, by adding a boolean value True/False to decide wether or not to keep it.

#### 2.1 Example of a filtering_prompt

« Your role is to rate the helpfulness of a document given a user message. The rating you can give is either True or False. 
Here's how to approach the task:

**Direct definition:** If the document explicitely defines the term or concept asked about in the user message, return 'True'.

**Mention Without Definition:** If the document mentions the term but does not provide a definition or explanation, return 'False'.

**Irrelevant Content:** If the document is unrelated to the user's query, or if the user's message is a greeting or conversational phrase (e.g. 'Hello'), return 'False'.

**When in doubt, Default to False:** If unsure, return 'False' by default. »

#### 2.2 Create a Filtering Prompt

**Create a filtering prompt to get only relevant info**

In [35]:
from langchain.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_cohere import CohereEmbeddings
from langchain_core.vectorstores import InMemoryVectorStore
from langchain import hub
from langchain_core.documents import Document
from langgraph.graph import START, StateGraph
from langchain.chat_models import init_chat_model
from typing_extensions import List, TypedDict

# Remplace l'URL par celle du blog que tu veux charger
url = "https://example.com/blog-post"

# Création du loader
loader = WebBaseLoader(url)

# Chargement des documents
docs = loader.load()

# Fractionnement du document
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,  # Taille d'un chunk
    chunk_overlap=100  # Chevauchement entre les chunks
)
all_splits = text_splitter.split_documents(docs)

# Création et stockage des embeddings
embeddings = CohereEmbeddings(model="embed-english-v3.0")
vector_store = InMemoryVectorStore(embeddings)
chunk_vector = vector_store.add_documents(documents=all_splits)  # Ajout des documents à la base vectorielle

# Définition du System Prompt
system_prompt = (
    "You are an AI assistant specialized in answering questions based on the provided context. "
    "Follow these rules: "
    "1) Answer in the same language as the user. "
    "2) Use only the given context to answer. "
    "3) If the context does not allow answering, say you don't know."
)

# Définition du Filtering Prompt
filtering_prompt = (
    "Your role is to rate the helpfulness of a document given a user message. The rating you can give is either True or False. "
    "Here's how to approach the task: "
    "**Direct definition:** If the document explicitly defines the term or concept asked about in the user message, return 'True'. "
    "**Mention Without Definition:** If the document mentions the term but does not provide a definition or explanation, return 'False'. "
    "**Irrelevant Content:** If the document is unrelated to the user's query, or if the user's message is a greeting or conversational phrase (e.g. 'Hello'), return 'False'. "
    "**When in doubt, Default to False:** If unsure, return 'False' by default."
)

# Définition du prompt pour le question-answering
prompt = hub.pull("rlm/rag-prompt")
llm = init_chat_model("command-r-plus", model_provider="cohere")


# Définition de l'état de l'application
class State(TypedDict):
    question: str
    context: List[Document]
    answer: str


# Définition des étapes de l'application
def retrieve(state: State):
    """
        Retrieve relevant documents from the vector store.
        Do a similarity search on the question in the state and retrieve the documents.
    """
    retrieved_docs = vector_store.similarity_search(state["question"])

    # Filtrage des documents
    filtered_docs = []
    for doc in retrieved_docs:
        prompt_input = f"{filtering_prompt}\nUser question: {state['question']}\nDocument: {doc.page_content}"
        relevance = llm.invoke(prompt_input)
        if relevance.content.strip().lower() == "true":
            filtered_docs.append(doc)

    return {"context": filtered_docs}


def generate(state: State):
    """
        Generate an answer using the retrieved context and the LLM.
        Format the retrieved documents into a single string to provide context.
    """
    docs_content = "\n\n".join(doc.page_content for doc in state["context"])
    messages = prompt.invoke({
        "system_prompt": system_prompt,
        "question": state["question"],
        "context": docs_content
    })
    response = llm.invoke(messages)
    return {"answer": response.content}


# Compilation et test de l'application
graph_builder = StateGraph(State).add_sequence([retrieve, generate])
graph_builder.add_edge(START, "retrieve")
graph = graph_builder.compile()

# Poser une question et obtenir une réponse
question_input = "Quel est le sujet principal de l'article ?"
result = graph.invoke({"question": question_input})

your_response = result["answer"]
your_sources = result["context"]  # Récupération des documents sources

print(your_response)  # Devrait être une chaîne de caractères

"""Display the sources used to get the answer"""
display(your_sources)


/Users/fabienribeiro/PycharmProjects/pythonProject/.venv/lib/python3.12/site-packages/langsmith/client.py:278: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


Le sujet principal de l'article est de discuter et d'analyser un sujet ou un événement particulier.


[]

### 3. The Rephrasing prompt

There is a 3rd prompt used for the retriever called the *rephrasing_prompt*.
It's used to reformat the user question in a well formatted prompt in order to perform the best similarity search possible
